# 1. 线性回归从零实现
- 了解细致的工作原理：
  - 方便自定义模型、自定义层或自定义损失函数
- 只使用张量&自动求导

In [2]:
%matplotlib inline 
import random 
import torch

## 1.1 生成数据集
- 使用低维数据
- 带有噪声
- 生成1000个样本：
  - 每个样本包含从正态分布中采样的2个特征
  - 合成数据集为一个矩阵$\mathbf{X}\in\mathbb{R}^{1000\times2}$

- 使用线性参数$\mathbf{w}=[2,-3.4]^\top$、$b=4.2$以及$\epsilon$生成数据集和标签
  - $\epsilon$服从均值为0的正态分布
  - 将标准差设为0.01

In [3]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
def synthetic_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

features, labels = synthetic_data(true_w, true_b, 1000)
print('features:', features[0],'\nlabel:', labels[0])

features: tensor([-0.7786,  0.3009]) 
label: tensor([1.6215])


## 1.2 读取数据集
- 训练模型时需要对数据集进行遍历，每次抽取一小批量样本，并使用其更新模型
- 需要定义一个函数：
  - 可以打乱数据集中的样本
  - 并以小批量方式获取数据
- 定义该函数为`data_iter`
  - 接收参数：批大小、特征矩阵和标签向量
  - 输出：大小为`batch_size`的小批量
- `yeild`关键字
  - `yield`与`return`的区别是，`return`会返回一个最终的值，并结束函数的执行，而`yield`可以返回多个值，并保持函数的状态
  - 可以用来在一个函数中返回一个生成器对象。生成器是一种特殊的函数，它不会一次性返回所有的值，而是每次返回一个值，然后暂停执行，直到下一次请求。
  - 当一个带有yield的函数被调用时，执行会停在`yield`语句处，并在生成器被迭代时从那里继续
- 注意：
  - 下述函数在实际应用中**执行效率很低**
  - 要求所有数据加载到内存并且执行大量随机内存访问

In [5]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)  # Shuffle list in place, and return None.
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor (
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

In [6]:
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y) 
    break

tensor([[ 0.4672, -2.2855],
        [-0.7969,  0.4304],
        [-0.9522, -0.4036],
        [-0.1783, -0.1045],
        [ 0.9622, -2.8179],
        [-0.3835, -0.5939],
        [-0.1761,  1.6146],
        [ 0.6210, -0.6852],
        [-0.5481, -0.9690],
        [-1.4299, -1.3684]]) 
 tensor([[12.8972],
        [ 1.1432],
        [ 3.6466],
        [ 4.1974],
        [15.7206],
        [ 5.4521],
        [-1.6411],
        [ 7.7589],
        [ 6.3909],
        [ 6.0013]])


## 1.3 初始化模型参数
- 在使用小批量随机梯度下降优化模型参数前，需要先有一些参数
  - 通过从均值为0、标准差为0.01的正态分布中采样随机数来初始化权重，并将偏置初始化为0
- 初始化参数后，更新它们，使得其足以拟合数据
  - 每次更新都要计算损失函数关于模型参数的梯度
  - 有该梯度，则可向减少损失的方向更新每个参数

In [9]:
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True) 
b = torch.zeros(1, requires_grad=True)

print("Initialized parameters:\n", "\tw = ", w, "\n\tb = ", b)

Initialized parameters:
 	w =  tensor([[-0.0035],
        [-0.0077]], requires_grad=True) 
	b =  tensor([0.], requires_grad=True)


## 1.4 定义模型及损失函数
- 定义模型：
  - 将模型的输入和参数同模型的输出关联起来
  - 计算输出：输入特征$\mathbf{X}$和模型权重$\mathbf{w}$的矩阵-向量乘法后再加上一个偏置$b$
- 定义损失函数：
  - 使用平方损失函数
  - 实现时，需要将真实值`y`的形状转换为和预测值`y_hat`的形状相同

In [10]:
def linreg(X, w, b):
    return torch.matmul(X, w) + b

def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

## 1.5 定义优化算法
